In [91]:
import numpy as np 
import pandas as pd 
import sqlite3 as sql 
import plotly.graph_objs as go 
import funciones as fn 
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors
from ipywidgets import interact 
#from mlxtend.preprocessing import TransactionEncoder 
from datetime import datetime 
 
# Sistemas de recomendación surprise 
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, SVD
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms import SlopeOne

import openpyxl
import 


In [64]:
conn = sql.connect('Data/movies2.db')
cur = conn.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('movies_m',),
 ('ratings_final',),
 ('full_ratings',),
 ('f_ratings',)]

In [72]:
# traer tabla de BD a python 
movies = pd.read_sql("select *  from movies_m", conn)
ratings = pd.read_sql('select * from ratings', conn)
ratingsf = pd.read_sql('select * from f_ratings', conn)

In [108]:
ratingsf

,user_id,movie_id,rating,timestamp,movie_title,movie_genres,clean_title,movie_year,fecha_nueva
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,2000-07-30
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995,2000-07-30
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,Heat,1995,2000-07-30
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,Seven,0,2000-07-30
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,"Usual Suspects, The",1995,2000-07-30
...,...,...,...,...,...,...,...,...,...
45420,609,589,3.0,847220990,Terminator 2: Judgment Day (1991),Action|Sci-Fi,Terminator 2: Judgment Day,1991,1996-11-05
45421,609,590,4.0,847220802,Dances with Wolves (1990),Adventure|Drama|Western,Dances with Wolves,1990,1996-11-05
45422,609,592,3.0,847220802,Batman (1989),Action|Crime|Thriller,Batman,1989,1996-11-05
45423,609,786,3.0,847221025,Eraser (1996),Action|Drama|Thriller,Eraser,1996,1996-11-05


In [104]:

def preprocesar():

    conn = sql.connect('Data/movies2.db')
    cur = conn.cursor()
    
    # convertir datos crudos a bases filtradas por usuarios que tengan cierto número de calificaciones
    fn.ejecutar_sql('preprocesamiento_n.sql', cur)

    # llevar datos que cambian constantemente a python
    movies = pd.read_sql('select * from movies_m', conn )
    ratings = pd.read_sql('select * from ratings', conn)
    f_ratings = pd.read_sql('select * from f_ratings', conn )
    usuarios = pd.read_sql('select distinct userId from ratings',conn)
  
    # transformación de datos crudos - Preprocesamiento
    movies['movie_year']=movies.movie_year.astype('int')

    ##### escalar para que año esté en el mismo rango ###
    sc=MinMaxScaler()
    movies[["year_sc"]]=sc.fit_transform(movies[['movie_year']])

    ## eliminar filas que no se van a utilizar ###
    movies_dum1=movies.drop(columns=['movie_id', 'movie_title', 'movie_genres'])


    return movies_dum1,movies,f_ratings, conn, cur

In [105]:
#########################################################################
###############Función para entrenar modelo por cada usuario ##########
###############Basado en contenido todo lo visto por el usuario Knn#############################

def recomendar(user_id):
    movies_dum1, movies,f_ratings, conn, cur = preprocesar()

    # Seleccionar solo los ratings del usuario seleccionado
    ratings = pd.read_sql('select * from ratings where userId=:user', conn, params={'user': user_id})
    l_movies_r = ratings['movieId'].to_numpy()

     # Renombrar las columnas para asegurar consistencia entre nombres
    movies_dum1 = movies_dum1.rename(columns={'user_id': 'userId'})
    movies = movies.rename(columns={'movie_id': 'movieId'})

    # Agregar la columna de movieID de la película a dummies para filtrar y mostrar nombre
    movies_dum1[['movieId', 'clean_title']] = movies[['movieId', 'clean_title']]

    # Filtrar películas calificadas por el usuario
    movies_r = movies_dum1[movies_dum1['movieId'].isin(l_movies_r)]
    movies_r = movies_r.drop(columns=['movieId', 'clean_title'])
    movies_r['indice'] = 1  # Para usar group by y que quede en formato pandas tabla de centroide
    centroide = movies_r.groupby('indice').mean()

    # Filtrar películas no vistas
    movies_nr = movies_dum1[~movies_dum1['movieId'].isin(l_movies_r)]
    movies_nr = movies_nr.drop(columns=['movieId', 'clean_title'])

    # Entrenar modelo
    model = neighbors.NearestNeighbors(n_neighbors=6, metric='cosine')
    model.fit(movies_nr)
    dist, idlist = model.kneighbors(centroide)

    ids = idlist[0]
    recomend_b = movies.loc[ids][['clean_title', 'movieId']]

    return recomend_b


In [109]:
## Función para obtener las 10 peliculas populares ####

def obtener_peliculas_populares(f_ratings):
    movies_dum1, movies,f_ratings, conn, cur = preprocesar()

    # Filtrar películas vistas por más de 20 personas y con calificación mayor a 4
    peliculas_populares = (f_ratings[f_ratings['rating'] > 4]
                           .groupby('movie_id')  # Cambiar a groupby con 'movieId'
                           .agg(avg_rat=('rating', 'mean'), movie_num=('rating', 'count'))
                           .reset_index())

    peliculas_populares = peliculas_populares.merge(movies[['movie_id', 'movie_title']], on='movie_id')

    # Filtrar las películas populares
    peliculas_populares = peliculas_populares[peliculas_populares['movie_num'] >= 20]
    peliculas_populares = peliculas_populares.sort_values(by=['avg_rat', 'movie_num'], ascending=False).head(10)

    return peliculas_populares[['movie_title', 'avg_rat', 'movie_num']]

In [110]:
##### Generar recomendaciones para usuario lista de usuarios ####
##### No se hace para todos porque es muy pesado #############
def main(list_user):

    recomendaciones_todos = pd.DataFrame()
    for user_id in list_user:

        recomendaciones = recomendar(user_id)
        recomendaciones["user_id"]=user_id
        recomendaciones.reset_index(inplace=True,drop=True)

        recomendaciones_todos=pd.concat([recomendaciones_todos, recomendaciones])

    recomendaciones_todos.to_excel('salidas/recomendaciones.xlsx')
    recomendaciones_todos.to_csv('salidas/recomendaciones.csv')

if __name__=="__main__":
    list_user=[1,37,100,609]
    main(list_user)


import sys
sys.executable

'c:\\Users\\USUARIO\\anaconda3\\python.exe'

In [112]:
def main_peliculas_populares():
    # Preprocesar los datos
    movies_dum1, movies, f_ratings, conn, cur = preprocesar()
    
    # Obtener las calificaciones y peliculas populares
    ratings = pd.read_sql('select * from f_ratings', conn)
    peliculas_populares = obtener_peliculas_populares(ratings)  # Solo pasamos ratings
    
    # Guardar las películas populares en un archivo Excel y csv
    peliculas_populares.to_excel('salidas/peliculas_populares.xlsx', index=False)
    peliculas_populares.to_csv('salidas/peliculas_populares.csv', index=False)

if __name__ == "__main__":
    main_peliculas_populares()